In [2]:
import pandas as pd
# import seaborn as sns
# from sklearn import tree
# import numpy as np


In [3]:
from subprocess import check_output
print(check_output(["ls", "data"]).decode("utf8"))

dataset_three.csv
numeric_one.csv
numeric_three_extended.csv
numeric_three_extended_normalized.csv
numeric_two_extrafeatures.csv
test.csv
train.csv
train_new_innerjoin.csv
train_new_leftjoin.csv
train_new_numeric.csv



In [4]:
# train_data = pd.read_csv("data/train_ready.csv")
train_data = pd.read_csv("data/numeric_one.csv")

In [5]:
train_data.shape

(108129, 14)

In [6]:
train_data.head(2)

,project_id,name,desc,goal,keywords,disable_communication,country,currency,deadline,state_changed_at,created_at,launched_at,backers_count,final_status
0,kkst1451568084,drawing for dollars,I like drawing pictures. and then i color them...,20.0,drawing-for-dollars,0.0,10,USD,1241333999,1241334017,1240600507,1240602723,3,1
1,kkst1474482071,Sponsor Dereck Blackburn (Lostwars) Artist in ...,"I, Dereck Blackburn will be taking upon an inc...",300.0,sponsor-dereck-blackburn-lostwars-artist-in-re...,0.0,10,USD,1242429000,1242432018,1240960224,1240975592,2,0


In [7]:
train_copy = train_data.copy()

# Feature Engineering - 

Some new features, using the unix timestamp featues.

In [8]:
# processing the UNIX timestamps into readable dates (pandas datetime):
cols = ['state_changed_at','created_at','launched_at','deadline']
for col in cols:
    train_data['%s' % col] = pd.to_datetime(train_data['%s' % col], unit='s')

# Project duration 

In [9]:
# dt - only work for datetime series - TypeError if the Series does not contain datetimelike values.
train_data['project_duration'] = (train_data.deadline - train_data.launched_at).dt.days

In [10]:
print(train_data.project_duration.mean())
print(train_data.project_duration.min())
print(train_data.project_duration.max())
print(train_data.project_duration[train_data.final_status == 1].mean())
print(train_data.project_duration[train_data.final_status == 0].mean())


33.9170342831
1
91
32.1244755649
34.7591479991


Not much significant - as overall mean (average days) is <b>34</b> days, while for successful projects it is <b>32</b> and failed it is <b>34</b>


# creation_to_launch:
a new feature to get the difference between creation and launch dates.

In [11]:
train_data['creation_to_launch'] = (train_data.launched_at - train_data.created_at).dt.days

In [12]:
print(train_data.creation_to_launch.mean())
print(train_data.creation_to_launch[train_data.final_status == 1].mean())
print(train_data.creation_to_launch[train_data.final_status == 0].mean())
print(train_data.creation_to_launch.max())
print(train_data.creation_to_launch.min())

37.2734049145
41.4838112323
35.2954273597
1903
0


Can be a helpful feature - as there's on average a week's difference between successful and failed projects.

 # description_size:
 size of the description string - alphabet count - can think over to make it better, if find any significance

In [13]:
train_data.insert(3, 'description_size', len(train_data.desc))
train_data.description_size = train_data.desc.str.len()

In [14]:
print(train_data.description_size.mean())
print(train_data.description_size[train_data.final_status ==1].mean())
print(train_data.description_size[train_data.final_status ==0].mean())

134.105502169
135.11339371
133.631960305


Completely insignificant - as no much difference in failed and successful projects' avergae descritpion size

In [15]:
train_data.shape

(108129, 17)

In [16]:
train_copy = train_data.copy()

In [17]:
# print all the data types of the features:
print(train_copy.dtypes)

project_id                       object
name                             object
desc                             object
description_size                float64
goal                            float64
keywords                         object
disable_communication           float64
country                           int64
currency                         object
deadline                 datetime64[ns]
state_changed_at         datetime64[ns]
created_at               datetime64[ns]
launched_at              datetime64[ns]
backers_count                     int64
final_status                      int64
project_duration                  int64
creation_to_launch                int64
dtype: object


# Saving - second level dataset 

In [28]:
train_copy.to_csv('data/numeric_two_extrafeatures.csv', index=False)  # index=False otherwise adds 'Unnamed' index

# Drop non-numeric features
mainly for neural network

In [18]:
# getting list of all non-numeric features.
nonnumeric_columns = train_copy.select_dtypes(['object']).columns
print(nonnumeric_columns)
# dropping the object type columns
train_copy.drop(nonnumeric_columns, axis=1, inplace=True)
print(train_copy.dtypes)


Index([u'project_id', u'name', u'desc', u'keywords', u'currency'], dtype='object')
description_size                float64
goal                            float64
disable_communication           float64
country                           int64
deadline                 datetime64[ns]
state_changed_at         datetime64[ns]
created_at               datetime64[ns]
launched_at              datetime64[ns]
backers_count                     int64
final_status                      int64
project_duration                  int64
creation_to_launch                int64
dtype: object


# Dropping timestamp/datetime features


In [19]:
train_copy.drop(['deadline', 'state_changed_at', 'created_at', 'launched_at'], axis=1, inplace=True)
print(train_copy.dtypes)


description_size         float64
goal                     float64
disable_communication    float64
country                    int64
backers_count              int64
final_status               int64
project_duration           int64
creation_to_launch         int64
dtype: object


# Fill empty cells - NaN


In [20]:
train_copy.description_size = train_copy.description_size.fillna(0)

# Train-Test Split 80-20

<b>train_test_split()</b>

Split arrays or matrices into random train and test subsets
Quick utility that wraps input validation and next(ShuffleSplit().split(X, y)) and application to input data into a single call for splitting (and optionally subsampling) data in a oneliner.

In [21]:
from sklearn.model_selection import train_test_split

class_label = train_copy.final_status.values.tolist()  # a list of class labels i.e. final_status 0 failed, 1 succesful.
del train_copy['final_status']  # removing the class/label column, so we can extract the test examples

features = train_copy.values.tolist()  # extracting features as lists.

# the split
X_train, X_test, y_train, y_test = train_test_split(features, class_label, test_size=0.2, random_state=40)



# 1.Decision tree

In [22]:
from sklearn import tree
import numpy as np

dtree_one = tree.DecisionTreeClassifier(criterion='entropy')  # getting the tree
# target = train_copy["final_status"].values
# features_named = train_copy[["goal", "backers_count", "disable_communication", "country"]].values

dtree_one = dtree_one.fit(X_train, y_train)  # fitting/training the tree classifier
# ------ for getting the exact entropy value per feature   ---------------------
# dtree_one = dtree_one.fit(train_copy[["goal", "backers_count", "disable_communication", "country", 
#                                       "description_size","creation_to_launch", "project_duration"]].values, 
#                           train_data["final_status"].values) 
# ------------------------------------------------------------------------------

dtree_one

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

# criterion : string, optional (default=”gini”)
The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain.


The feature importances. The higher, the more important the feature. The importance of a feature is computed as the (normalized) total reduction of the criterion brought by that feature. It is also known as the <b>Gini importance</b> [R259].


One way to quickly see the result of your decision tree is to see the importance of the features that are included. This is done by requesting the <b>.feature_importances_</b> attribute of your tree object. Another quick metric is the mean accuracy that you can compute using the <b>.score()</b> function with features_one and target as arguments.


In [23]:
# Look at the importance and score of the included features
print(dtree_one.feature_importances_)  # probably be calculating the inforamtion gain/entropy

# goal, backers_count, disable_communication, country, description_size, creation_to_launch, project_duration
# [ 0.21459478  0.48072188  0.00098577  0.01294921  0.11153803  0.10499667 0.07421366]

[ 0.11217714  0.21589638  0.00073401  0.01216421  0.48066605  0.07196603
  0.10639618]


In [24]:
# test accuracy 
dtree_one.score(X_test, y_test)

0.82904836770553958

# control overfitting - tuned decision tree

Control overfitting by setting "max_depth" to 10 and "min_samples_split" to 5 : my_tree_two

<b>max_depth</b> = 10 (default None) - The maximum depth of the tree. If None, then nodes are expanded until all leaves are pure or until all leaves contain less than min_samples_split samples.

<b>min_samples_split</b> = 5 (default 2) - The minimum number of samples required to split an internal node:
 - If int, then consider min_samples_split as the minimum number.
 - If float, then min_samples_split is a percentage and ceil(min_samples_split * n_samples) are the minimum number of samples for each split.

my_tree_two = tree.DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, random_state = 1)


In [25]:
max_depth = 7
min_samples_split = 5

dtree_two = tree.DecisionTreeClassifier(max_depth = max_depth, min_samples_split = min_samples_split, random_state = 1, criterion='entropy')
dtree_two = dtree_two.fit(X_train, y_train)  # fitting/training the tree classifier

print(dtree_two.feature_importances_)
# goal, backers_count, disable_communication, country, description_size, creation_to_launch, project_duration
# [ 0.21459478  0.48072188  0.00098577  0.01294921  0.11153803  0.10499667 0.07421366]

[  4.08933456e-04   2.96050136e-01   6.59989556e-04   4.07275008e-04
   7.01016754e-01   1.19606427e-03   2.60847791e-04]


In [26]:
print(dtree_two.score(X_test, y_test))

0.874040506797


# Tree 1:
 - score: <b>0.99382219386103632</b>
 - feature importance (gini): [ 0.21666469,  0.55074331,  0.14067283,  0.09191916]
 
# Tree 2:
 - score: <b>0.99382219386103632</b>
 - feature importance (entropy): [ 0.23237351,  0.5140295,   0.15209979,  0.10149719]



# 2. RandomForest Classifier

the Random Forest technique handles the overfitting problem you faced with decision trees. It grows multiple (very deep) classification trees using the training set. <b>At the time of prediction, each tree is used to come up with a prediction and every outcome is counted as a vote</b>. For example, if you have trained 3 trees with 2 saying a passenger in the test set will survive and 1 says he will not, the passenger will be classified as a survivor. This approach of overtraining trees, but having the majority's vote count as the actual classification decision, avoids overfitting.


bagging and random forests - Averages predictions from a base set of models, can have difficulty making predictions near 0 and 1 because variance in the underlying base models will bias predictions that should be near zero or one away from these values.

In [28]:
from sklearn.ensemble import RandomForestClassifier

# features = train_copy[["goal", "goal_log", "backers_count", "creation_to_launch", "project_duration"]].values
# target = train_copy["final_status"].values

# Building and fitting my_forest
forest = RandomForestClassifier(max_depth = 10, min_samples_split=2, n_estimators = 100, random_state = 1) # initializing classifier
my_forest = forest.fit(X_train, y_train)  # fitting/training the classifier

# Print the score of the fitted random forest
print(my_forest.score(X_test, y_test))   # for calculating the mean accuracy of the model.





0.873439378526


In [29]:
my_forest.feature_importances_
# goal, backers_count, disable_communication, country, description_size, creation_to_launch, project_duration


array([ 0.01231406,  0.16131121,  0.00103823,  0.00549291,  0.77880405,
        0.02037438,  0.02066516])

# 3. kNN

In [30]:
from sklearn import neighbors

knn = neighbors.KNeighborsClassifier()
knn_trained = knn.fit(X_train, y_train)

print(knn_trained.score(X_test, y_test))

0.851706279478


# 4. A basic deep learning using keras - uses theano and tensorflow

In [33]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras.layers import advanced_activations  # all the available activation functions e.g. relu, 


Using TensorFlow backend.


To obtain better accuracy we need to make specific choices for the number of 
- epochs of training, 
- the mini-batch size, and 
- the learning rate.

The above params are known as hyper-parameters for our neural network, in order to distinguish them from the parameters (weights and biases) learnt by our learning algorithm. If we choose our hyper-parameters poorly, we can get bad results. Suppose, for example, that we'd chosen the learning rate to be η=0.001 η=0.001,



In [53]:
# the basic params for deep learning network
FEATURE_NUM = 7   # the input dimension - no of features == input layer nodes.
CLASSES = 1  # binary classification problem - successful 1 or failed 0 projects - has to be one i.e. size of output layer.
# HIDDEN1_SIZE = 100
# HIDDEN2_SIZE = 50
HIDDEN1_SIZE = 50
HIDDEN2_SIZE = 25
MAX_RANGE = 100



These models have a number of methods in common:

<b>model.summary()</b>: prints a summary representation of your model

<b>model.get_config()</b>: returns a dictionary containing the configuration of the model. 

The model can be reinstantiated from its config via:
config = model.get_config()
model = Model.from_config(config)
 or, for Sequential:
model = Sequential.from_config(config)


<b>model.get_weights()</b>: returns a list of all weight tensors in the model, as Numpy arrays.

<b>model.set_weights(weights)</b>: sets the values of the weights of the model, from a list of Numpy arrays. The arrays in the list should have the same shape as those returned by get_weights().

In [54]:
model = Sequential()  # The Sequential model is a linear stack of layers.

<b>model = Sequential([
    Dense(32, input_shape=(784,)),
    Activation('relu'),
    Dense(10),
    Activation('softmax'),
])</b>


You can also simply add layers via the <b>.add()</b> method:

model = Sequential

model.add(Dense(32, input_dim=784

model.add(Activation('relu'))

In [55]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Total params: 0
Trainable params: 0
Non-trainable params: 0
_________________________________________________________________


Both the following methods return - the representation that does not include the weights, only the architecture.

<b>model.to_json()</b>: returns a representation of the model as a JSON string. 

<b>model.to_yaml()</b>: returns a representation of the model as a YAML string. 

In [56]:
# model.to_json()

Specifying the input shape

The model needs to know what input shape it should expect. For this reason, the first layer in a Sequential model (and only the first, because following layers can do automatic shape inference) needs to receive information about its input shape. There are several possible ways to do this:

 - Pass an <b>input_shape</b> argument to the first layer. This is a shape tuple (a tuple of integers or None entries, where None indicates that any positive integer may be expected). In<b> input_shape</b>, the batch dimension is not included.

 - Some 2D layers, such as <b>Dense</b>, support the specification of their input shape via the argument <b>input_dim</b>, and some 3D temporal layers support the arguments <b>input_dim</b> and <b>input_length</b>.

 - If you ever need to specify a fixed batch size for your inputs (this is useful for stateful recurrent networks), you can pass a  batch_size argument to a layer. If you pass both <b>batch_size=32</b> and <b>input_shape=(6, 8)</b> to a layer, it will then expect every batch of inputs to have the batch shape <b>(32, 6, 8)</b>.

In [57]:
model.add(Dense(HIDDEN1_SIZE, input_dim=FEATURE_NUM, init='uniform'))  # next to input layer.
# model.add(Dense(10, input_dim=6, init='uniform'))  # next to input layer.

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(50, kernel_initializer="uniform", input_dim=7)`
  if __name__ == '__main__':


In [58]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                400       
Total params: 400
Trainable params: 400
Non-trainable params: 0
_________________________________________________________________


model = Sequential()

Dense(64) is a fully-connected layer with 64 hidden units.
<b>in the first layer, you must specify the expected input data shape</b>, here, 20-dimensional vectors.

model.add(Dense(64, activation='relu', input_dim=20))

model.add(Dropout(0.5))

model.add(Dense(64, activation='relu'))

model.add(Dropout(0.5))

model.add(Dense(10, activation='softmax'))

# activation:

Activations can either be used through an Activation layer, or through the activation argument supported by all forward layers:




# Available activations
<b>softmax</b> - softmax(x, axis=-1)

Softmax activation function.

Arguments

x : Tensor. - axis: Integer, axis along which the softmax normalization is applied.

Returns

Tensor, output of softmax transformation.

Raises

    ValueError: In case dim(x) == 1.

<b>elu</b> - elu(x, alpha=1.0)

<b>selu</b> - selu(x)

Scaled Exponential Linear Unit. (Klambauer et al., 2017)

Arguments

    x: A tensor or variable to compute the activation function for.

References

    Self-Normalizing Neural Networks

<b>softplus</b> - softplus(x)

<b>softsign</b> - softsign(x)

<b>relu</b> - relu(x, alpha=0.0, max_value=None)

<b>tanh</b> - tanh(x)

<b>sigmoid</b> - sigmoid(x)

<b>hard_sigmoid</b> - hard_sigmoid(x)

<b>linear</b> - linear(x)


In [59]:
model.get_config()

[{'class_name': 'Dense',
  'config': {'activation': 'linear',
   'activity_regularizer': None,
   'batch_input_shape': (None, 7),
   'bias_constraint': None,
   'bias_initializer': {'class_name': 'Zeros', 'config': {}},
   'bias_regularizer': None,
   'dtype': 'float32',
   'kernel_constraint': None,
   'kernel_initializer': {'class_name': 'RandomUniform',
    'config': {'maxval': 0.05, 'minval': -0.05, 'seed': None}},
   'kernel_regularizer': None,
   'name': 'dense_4',
   'trainable': True,
   'units': 50,
   'use_bias': True}}]

In [60]:
model.add(advanced_activations.ELU(alpha=1.0))  # adds activation to the last specified layer 
model.add(Dropout(0.6))
model.add(Dense(HIDDEN2_SIZE, init='uniform'))  # # after the first layer, you don't need to specify

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(25, kernel_initializer="uniform")`
  app.launch_new_instance()


In [61]:
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                400       
_________________________________________________________________
elu_3 (ELU)                  (None, 50)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
Total params: 1,675
Trainable params: 1,675
Non-trainable params: 0
_________________________________________________________________


In [62]:
# the output layer - for binary classification it has 1 node.

model.add(advanced_activations.ELU(alpha=1.0))
model.add(Dropout(0.6))
model.add(Dense(CLASSES, init='uniform', activation='softmax'))  # output layer - NUmber of classes

/home/nahmed/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(1, activation="softmax", kernel_initializer="uniform")`


In [63]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                400       
_________________________________________________________________
elu_3 (ELU)                  (None, 50)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
_________________________________________________________________
elu_4 (ELU)                  (None, 25)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 26        
Total para

# Training persistence - saving the minimized weights

<b>model.save_weights(filepath)</b>: saves the weights of the model as a HDF5 file.

<b>model.load_weights(filepath, by_name=False)</b>: loads the weights of the model from a HDF5 file (created by  save_weights). 

By default, the architecture is expected to be unchanged. 

To load weights into a different architecture (with some layers in common), use by_name=True to load only those layers with the same name.

In [64]:
checkpointer = ModelCheckpoint(filepath="/tmp/weights_data_two.hdf5",
                               verbose=1,
                               save_best_only=True)

In [65]:
tensorboard = TensorBoard(log_dir='./logs',
                          histogram_freq=0,
                          write_graph=True,
                          write_images=False)

In [66]:
# tensorboard.

Before training a model, you need to configure the learning process, which is done via the <b>compile</b> method. It receives three arguments:

   - <b>An optimizer</b>. This could be the string identifier of an existing optimizer (such as rmsprop or adagrad), or an instance of the Optimizer class. See: optimizers.
   - <b>A loss function</b>. This is the objective that the model will try to minimize. It can be the string identifier of an existing loss function (such as<b> categorical_crossentropy</b> or <b>mse</b>), or it can be an objective function. See: losses.
   - A list of metrics. For any classification problem you will want to set this to metrics=['accuracy']. A metric could be the string identifier of an existing metric or a custom metric function.


# Optimizers: 
https://keras.io/optimizers/

- Stochastic Gradient Descent - 'sgd' -
- Adam - 'Adam' - a stochastic optimizer.  an algorithm for first-order gradient-based optimization of stochastic objective functions, based on adaptive estimates of lower-order moments. The method is straightforward to implement, is computationally efficient, has little memory requirements, is invariant to diagonal rescaling of the gradients, and is well suited for problems that are large in terms of data and/or parameters.

Optimizers:
Arguments

- lr: float >= 0. Learning rate.
- momentum: float >= 0. Parameter updates momentum.
- decay: float >= 0. Learning rate decay over each update.
- nesterov: boolean. Whether to apply Nesterov momentum.



# Metrics:

A metric is a function that is used to judge the performance of your model. Metric functions are to be supplied in the metrics parameter when a model is compiled.
<b>A metric function is similar to an loss function, except that the results from evaluating a metric are not used when training the model.</b>




from keras import metrics

model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=[metrics.mae, metrics.categorical_accuracy,<b> metrics.binary_accuracy</b])




In [67]:
model.compile(loss='binary_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])


In [68]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 50)                400       
_________________________________________________________________
elu_3 (ELU)                  (None, 50)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 25)                1275      
_________________________________________________________________
elu_4 (ELU)                  (None, 25)                0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 25)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 26        
Total para

In [69]:
# the number of layers added to the model:
model.layers

# Fitting the model:
Keras models are trained on Numpy arrays of input data and labels. For training a model, you will typically use the <b>fit</b> function. 

epoch - 

callbacks:


# For a single-input model with 2 classes (binary classification):

model = Sequential()

model.add(Dense(32, activation='relu', input_dim=100))

model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])
              
              
              
              
# For a single-input model with 10 classes (categorical classification):

model = Sequential

model.add(Dense(32, activation='relu', input_dim=100))

model.add(Dense(10, activation='softmax'))

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

####generate dummy data

import numpy as np
data = np.random.random((1000, 100))
labels = np.random.randint(10, size=(1000, 1))

####Convert labels to categorical one-hot encoding

one_hot_labels = keras.utils.to_categorical(labels, num_classes=10)

####Train the model, iterating on the data in batches of 32 samples

model.fit(data, one_hot_labels, epochs=10, batch_size=32)

# fit()
Trains the model for a fixed number of epochs.

Arguments

- <b>x</b>: input data, as a Numpy array or list of Numpy arrays (if the model has multiple inputs).
- <b>y</b>: labels, as a Numpy array.
- <b>batch_size</b>: integer. Number of samples per gradient update.
- <b>epochs</b>: integer, the number of epochs to train the model.
- <b>verbose</b>: 0 for no logging to stdout, 1 for progress bar logging, 2 for one log line per epoch.
- <b>callbacks</b>: list of keras.callbacks.Callback instances. List of callbacks to apply during training. See callbacks.
- <b>validation_split</b>: float (0. < x < 1). Fraction of the data to use as held-out validation data.
- <b>validation_data</b>: tuple (x_val, y_val) or tuple (x_val, y_val, val_sample_weights) to be used as held-out validation data. Will override validation_split.
- <b>shuffle</b>: boolean or str (for 'batch'). Whether to shuffle the samples at each epoch. 'batch' is a special option for dealing with the limitations of HDF5 data; it shuffles in batch-sized chunks.
- <b>class_weight</b>: dictionary mapping classes to a weight value, used for scaling the loss function (during training only).
- <b>sample_weight</b>: Numpy array of weights for the training samples, used for scaling the loss function (during training only). You can either pass a flat (1D) Numpy array with the same length as the input samples (1:1 mapping between weights and samples), or in the case of temporal data, you can pass a 2D array with shape (samples, sequence_length), to apply a different weight to every timestep of every sample. In this case you should make sure to specify sample_weight_mode="temporal" in compile().
- <b>initial_epoch</b>: epoch at which to start training (useful for resuming a previous training run)


In [70]:
# Epoch 100, batch size 50, optimizer Adam, loss binary_crossentropy
model.fit(X_train, y_train,
#           nb_epoch =MAX_RANGE,  # 100 number of iteration 
          nb_epoch=10,
#           batch_size=1000,
          batch_size=50,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

Train on 86503 samples, validate on 21626 samples
Epoch 1/10
86503/86503 [==============================] - 7s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 2/10
86503/86503 [==============================] - 8s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 3/10
86503/86503 [==============================] - 7s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 4/10
86503/86503 [==============================] - 6s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 5/10
86503/86503 [==============================] - 6s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 6/10
86503/86503 [==============================] - 5s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 7/10
86503/86503 [==============================] - 5s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 8/10
86503/86503 [=======================

# Understanding the output:

<b>Epoch</b>: A full pass over all of your training data.

For example, we have 86503 enteries in the training data. So an epoch concludes when it has finished a training pass over all 86503 observations.

<b>Loss</b>: A scalar value that we attempt to minimize during our training of the model. The lower the loss, the closer our predictions are to the true labels (the higher the model accuracy). The commonly user loss estimators are:
- Mean Squared Error (MSE) - 'mean_squared_error'
- Mean Absolute Error (MAE) - 'mean_absolute_error'
- Categorical Cross Entropy - 'categorical_crossentropy'
- Binary cross entropy - 'binary_crossentropy'

# ---------------------------------------------
What you'd expect to see from running fit on your Keras model, is a decrease in loss over n number of epochs. Your training run is rather abnormal, as your loss is actually increasing. This could be due to a learning rate that is too large, which is causing you to overshoot optima.

# ----------------------------------------------
Ways to improve your model:

- Use cross validation in your Keras model in order to find out how the model actually perform, does it generalize well when predicting new data it has never seen before?
- Adjust your learning rate, structure of neural network model, number of hidden units / layers, init, optimizer, and activator parameters used in your model among myriad other things.

Combining sklearn's GridSearchCV with Keras can automate this process.


In [121]:
score, accuray = model.evaluate(X_test, y_test, batch_size=50)  # separate evaluation call.

21450/21626 [============================>.] - ETA: 0s

In [122]:
score

10.790191331073334

In [125]:
accuray


0.32317580754404363

In [124]:
model.get_weights()

[array([[ -1.37616880e-02,   4.44565080e-02,   4.51512448e-02,
          -1.20556727e-02,  -4.63260189e-02,   4.86928225e-03,
          -4.09003124e-02,  -5.19101694e-03,   4.49616797e-02,
          -1.01764686e-02],
        [ -2.13987827e-02,  -2.45790724e-02,   3.56110372e-02,
          -3.77490893e-02,   4.84151766e-03,   1.60002448e-02,
           1.76310204e-02,  -2.07070950e-02,   1.36582963e-02,
          -4.51361798e-02],
        [  2.99240611e-02,  -1.16293915e-02,   3.07920016e-02,
          -4.08664346e-05,   2.12794803e-02,  -1.40939951e-02,
           1.08783953e-02,  -4.65467460e-02,  -1.96858533e-02,
          -4.70530242e-04],
        [ -6.63654879e-03,  -3.03328764e-02,   4.06140946e-02,
          -8.30323622e-03,  -4.50867899e-02,  -3.04585453e-02,
           2.40007304e-02,  -4.36535589e-02,   1.43903755e-02,
           4.63988222e-02],
        [  2.10049488e-02,   2.06062905e-02,  -3.13359387e-02,
           2.95282044e-02,   2.19012983e-02,  -2.70376354e-03,
      

# ---------------------------------------------------------------------------------------------------
Epoch 10

Batch Size - 100

Loss 'binary_crossentropy'

optimizer='Adam'
           

In [127]:
model.fit(X_train, y_train,
          nb_epoch=10,
          batch_size=100,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

Train on 86503 samples, validate on 21626 samples
Epoch 1/10
86503/86503 [==============================] - 2s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 2/10
86503/86503 [==============================] - 3s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 3/10
86503/86503 [==============================] - 3s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 4/10
86503/86503 [==============================] - 2s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 5/10
86503/86503 [==============================] - 3s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 6/10
86503/86503 [==============================] - 3s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 7/10
86503/86503 [==============================] - 3s - loss: 10.8609 - acc: 0.3187 - val_loss: 10.7902 - val_acc: 0.3232
Epoch 8/10
86503/86503 [=======================

In [128]:
score, accuray = model.evaluate(X_test, y_test, batch_size=50)  # separate evaluation call.

20050/21626 [==========================>...] - ETA: 0s

In [130]:
accuray

0.32317580754404363

# ----------------------------------------------------------------------------------------------
Epoch - 10 - as even after the first iteration the loss value didn't imporved (minimized).

Batch Size - 100

Optimizer - Stochastic Gradient Descent - sgd

Loss - mean_squared_error

In [131]:
model.compile(loss='mean_squared_error',
              optimizer='sgd',
              metrics=['accuracy'])

In [132]:
model.fit(X_train, y_train,
          nb_epoch=10,
          batch_size=100,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

Train on 86503 samples, validate on 21626 samples
Epoch 1/10
86503/86503 [==============================] - 3s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 2/10
86503/86503 [==============================] - 2s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 3/10
86503/86503 [==============================] - 3s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 4/10
86503/86503 [==============================] - 2s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 5/10
86503/86503 [==============================] - 2s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 6/10
86503/86503 [==============================] - 3s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 7/10
86503/86503 [==============================] - 3s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 8/10
86503/86503 [==============================] - 3s 

In [133]:
score, accuray = model.evaluate(X_test, y_test, batch_size=50)  # separate evaluation call.

20500/21626 [===========================>..] - ETA: 0s

In [134]:
accuray

0.32317580754404363

# ------------------------------------------------------------------------------------------------------

- Optimizer - SGD
- learning rate - 0.01
- Epoch - 5
- Batch size - 1000 


Optimizers:
Arguments

- lr: float >= 0. Learning rate.
- momentum: float >= 0. Parameter updates momentum.
- decay: float >= 0. Learning rate decay over each update.
- nesterov: boolean. Whether to apply Nesterov momentum.



In [136]:
from keras import optimizers

sgd = optimizers.SGD(lr=0.01, clipvalue=0.5)

In [137]:
model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['accuracy'])

In [138]:
model.fit(X_train, y_train,
          nb_epoch=5,
          batch_size=1000,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

Train on 86503 samples, validate on 21626 samples
Epoch 1/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 2/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 3/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 4/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 5/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232


# ----------------------------------------------------------------------------------------------------

- Optimizer - SGD
- learning rate - 0.001
- Epoch - 5
- Batch size - 1000 

In [139]:
sgd = optimizers.SGD(lr=0.001, clipvalue=0.5)
model.compile(loss='mean_squared_error',
              optimizer=sgd,
              metrics=['accuracy'])

In [140]:
model.fit(X_train, y_train,
          nb_epoch=5,
          batch_size=1000,
          validation_data=(X_test, y_test),
          callbacks=[checkpointer, tensorboard])

Train on 86503 samples, validate on 21626 samples
Epoch 1/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 2/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 3/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 4/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
Epoch 5/5
86503/86503 [==============================] - 0s - loss: 0.6813 - acc: 0.3187 - val_loss: 0.6768 - val_acc: 0.3232
